In [1]:
! pip install -q kaggle tensorflow numpy pandas scikit-learn

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d deadskull7/fer2013

print("\n--- Unzipping and Verification ---")
! unzip -o fer2013.zip
! rm fer2013.zip

print("\n--- Directory Check ---")
! ls

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/deadskull7/fer2013
License(s): CC0-1.0
  0% 0.00/96.6M [00:00<?, ?B/s]
100% 96.6M/96.6M [00:00<00:00, 1.11GB/s]

--- Unzipping and Verification ---
Archive:  fer2013.zip
  inflating: fer2013.csv             

--- Directory Check ---
fer2013.csv  kaggle.json  sample_data


In [3]:
# Cell 2: Model Training and Saving (Optimized for Speed)

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

DATASET_PATH = 'fer2013.csv'
IMG_SIZE = 48
NUM_CLASSES = 7
BATCH_SIZE = 128
EPOCHS = 50
MODEL_NAME = 'facial_emotion_cnn_model_FAST.h5'

print(f"Loading data from path: {DATASET_PATH}...")
data = pd.read_csv(DATASET_PATH)
print("Data successfully loaded!")

pixels = data['pixels'].tolist()
emotions = data['emotion'].values

X = []
for pixel_sequence in pixels:
    face = np.fromstring(pixel_sequence, dtype=int, sep=' ').reshape(IMG_SIZE, IMG_SIZE)
    X.append(face)

X = np.array(X).astype('float32') / 255.0
X = np.expand_dims(X, -1)
y = to_categorical(emotions, num_classes=NUM_CLASSES)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=emotions)

print(f"Training Data Shape: {X_train.shape}")

def create_fast_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(32, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (IMG_SIZE, IMG_SIZE, 1)
model = create_fast_cnn_model(input_shape, NUM_CLASSES)

datagen = ImageDataGenerator(
    rotation_range=15, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\n--- Starting Model Training (Optimized for Presentation) ---")
model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

model.save(MODEL_NAME)
print(f"\nFINAL MODEL SAVED: {MODEL_NAME}")

Loading data from path: fer2013.csv...
Data successfully loaded!
Training Data Shape: (28709, 48, 48, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Starting Model Training (Optimized for Presentation) ---
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


224/224 ━━━━━━━━━━━━━━━━━━━━ 28s 100ms/step - accuracy: 0.2251 - loss: 1.8547 - val_accuracy: 0.2561 - val_loss: 1.7919
Epoch 2/50
  1/224 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2344 - loss: 1.8106

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2344 - loss: 1.8106 - val_accuracy: 0.2581 - val_loss: 1.7914
Epoch 3/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 15s 67ms/step - accuracy: 0.2461 - loss: 1.7982 - val_accuracy: 0.2673 - val_loss: 1.7590
Epoch 4/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2969 - loss: 1.7322 - val_accuracy: 0.2672 - val_loss: 1.7584
Epoch 5/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - accuracy: 0.2682 - loss: 1.7748 - val_accuracy: 0.3222 - val_loss: 1.6894
Epoch 6/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3438 - loss: 1.6378 - val_accuracy: 0.3190 - val_loss: 1.6917
Epoch 7/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 15s 67ms/step - accuracy: 0.2925 - loss: 1.7387 - val_accuracy: 0.3650 - val_loss: 1.6156
Epoch 8/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2891 - loss: 1.7727 - val_accuracy: 0.3628 - val_loss: 1.6167
Epoch 9/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - accuracy: 0.3137 - loss: 1.7045 - val_accuracy: 0.393


FINAL MODEL SAVED: facial_emotion_cnn_model_FAST.h5
